<a href="https://colab.research.google.com/github/NagypalMarton/Speaking-with-LLM-in-mother-tongue/blob/main/voice_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet transformers accelerate torch gradio faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import gradio as gr

In [3]:
# 3. GPU detektálása
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [4]:
# 4. Dokumentumok
documents = [
    "Google Colab egy egyszerű felhős Jupyter notebook környezet, ahol GPU is elérhető.",
    "A Gradio egy Python könyvtár, amely gyorsan lehetővé teszi AI modellek webes UI-jának létrehozását.",
    "A RAG technológia ötvözi a kereső (retriever) és generatív (generator) modelleket, hogy pontosabb válaszokat adjon.",
    "A Hugging Face modellek könnyen letölthetők és használhatók különböző NLP feladatokra.",
    "A phi-4 egy erős, open-source nyelvi modell a Microsofttól, amely képes természetes nyelvi kérdésekre válaszolni."
]

In [5]:
# 5. Embedding modell és indexelés
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(documents)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# 6. LLM betöltése (Phi-4-mini-instruct)
model_id = "microsoft/phi-4-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.91M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [12]:
#Zero Shot Intent Detection
def detect_intent_llm(user_utterance, lang="hu"):
    """
    Zero-shot intent detection LLM-mel.
    user_utterance: a felhasználó mondata
    lang: 'hu' vagy 'en'
    Visszaadott intent: szöveg (pl. 'foglalás', 'info', stb.)
    """
    if lang == "hu":
        prompt = (
            "Az alábbi mondat alapján válaszd ki a szándékot az alábbiak közül (csak egy szót adj vissza, szóköz nélkül): "
            "foglalás, lemondás, módosítás, információ, köszönés, egyéb.\n"
            f"Felhasználó: {user_utterance}\nSzándék:"
        )
    else:
        prompt = (
            "Based on the following sentence, choose the intent from: booking, cancel, reschedule, info, greeting, other. "
            "Return only one word, no spaces.\n"
            f"User: {user_utterance}\nIntent:"
        )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=3)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Szándék kiszűrése a válaszból
    intent = answer.split(":")[-1].strip().split()[0]
    return intent

In [13]:
def rag_phi4mini_chatbot(user_utterance, lang="hu"):
    # 1. Intent detektálás
    intent = detect_intent_llm(user_utterance, lang=lang)
    # 2. Embed és keresés (RAG)
    q_embed = embedder.encode([user_utterance])
    D, I = index.search(np.array(q_embed), k=1)
    context = documents[I[0][0]]
    # 3. RAG threshold
    if D[0][0] > 1.0:
        # Nincs releváns kontextus, csak a kérdést adom a modellnek
        prompt = f"Kérdés: {user_utterance}\nSzándék: {intent}\nVálasz:"
    else:
        # Van kontextus, azt is beadom
        prompt = f"""
Az alábbi kontextus alapján válaszolj a kérdésre magyarul:

KONTEKSTUS:
{context}

KÉRDÉS:
{user_utterance}

SZÁNDÉK:
{intent}

VÁLASZ:
"""
    # 4. LLM generálás
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Csak a válasz rész visszaadása
    if "VÁLASZ:" in answer:
        answer = answer.split("VÁLASZ:")[-1].strip()
    elif "Válasz:" in answer:
        answer = answer.split("Válasz:")[-1].strip()
    return answer, intent

In [14]:
# 8. Gradio UI
iface = gr.Interface(
    fn=rag_phi4mini_chatbot,
    inputs=gr.Textbox(lines=2, label="Kérdés magyarul"),
    outputs=gr.Textbox(label="Chatbot válasza"),
    title="RAG Chatbot - Microsoft Phi-4-mini-instruct + Gradio (Google Colab, GPU)",
    description="Kérdezz bátran! A chatbot a kontextus alapján válaszol magyarul."
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d47fc291bde29af06e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
